In [187]:
#Adam Bloodgood

import requests
import json
import pandas as pd
import string
import math

In [133]:
# Gather details from all of the shots from a specific game ID.

def getShotInfo(gameId):

    # Get play-by-play data from the selected game and filter out just the individual plays.
    play_by_play = requests.get(f"https://api-web.nhle.com/v1/gamecenter/{gameId}/play-by-play")
    play_by_play = play_by_play.json()
    play_by_play['plays']

    # Filter out plays that are either shots or goals
    event_types = ["shot-on-goal", "goal"]

    data = pd.DataFrame.from_dict(play_by_play['plays'])
    shots = data[data['typeDescKey'].isin(event_types)]

    # Clean up dataframe to make sense
    shot_details = pd.json_normalize(shots['details'])
    shot_results = shots['typeDescKey'].reset_index()
    shot_details = pd.concat([shot_results, shot_details], axis=1)
    del shot_details['index']

    return shot_details


In [199]:
# Retrieve a goalie's name based on their player ID.

def getGoalieName(goalieId):
    goalie_info = requests.get(f"https://api-web.nhle.com/v1/player/{goalieId}/landing")

    # Make sure the shot was not on an empty net
    if goalie_info.status_code != 404:
        goalie_info = goalie_info.json()
        
        
        first_name = goalie_info['firstName']['default']
        last_name = goalie_info['lastName']['default']

        return f"{last_name}, {first_name}"
    return

In [200]:
# Filter which shots were considered deflections or tip-ins

def getDeflections(shot_details):
    event_types = ["tip-in", "deflected"]
    deflections = shot_details[shot_details['shotType'].isin(event_types)]

    return deflections

In [ ]:
goalie_db = {}

SEASON = 2023
SEASON_TYPE = 2 # 1 = preseason, 2 = regular season, 3 = playoffs
GAMES_TRACKING = 1312
START_GAME = 1

for game in range(START_GAME, START_GAME + GAMES_TRACKING):
    
    # Insert appropriate number of zeros to get the right play-by-play url
    game_code = game
    if game < 10:
        game_code = f"000{game}"
    elif game < 100:
        game_code = f"00{game}"
    elif game < 1000:
        game_code = f"0{game}"

    gameId = f"{SEASON}0{SEASON_TYPE}{game_code}"

    shot_info = getShotInfo(gameId)

    deflections = getDeflections(shot_info)

    #Go through every shot during the game. If the goalie isn't in the dictionary, add them. Then add shots and goals.
    for index, shot in deflections.iterrows():
        goalie_id = shot['goalieInNetId'] 
        if math.isnan(goalie_id) == False:
            goalie_id = int(goalie_id)
            goalie_name = getGoalieName(goalie_id)
            if goalie_name not in goalie_db:
                goalie_db[goalie_name] = {'deflections' : 0, 'goals': 0, 'goalie_id': goalie_id}

            if shot['typeDescKey'] == 'goal':
                goalie_db[goalie_name]['goals'] += 1
                
            goalie_db[goalie_name]['deflections'] += 1

    #Calculate saves and sv% for each goalie
    for goalie in goalie_db.keys():
        goalie_db[goalie]["saves"] = goalie_db[goalie]['deflections'] - goalie_db[goalie]['goals'] 
        goalie_db[goalie]["SV%"] = round(goalie_db[goalie]['saves'] / goalie_db[goalie]['deflections'], 3)

        


In [205]:
#reorder values
for goalie in goalie_db.keys():

    goalie_db[goalie]["saves"] = goalie_db[goalie]['deflections'] - goalie_db[goalie]['goals'] 
    goalie_db[goalie]["SV%"] = round(goalie_db[goalie]['saves'] / goalie_db[goalie]['deflections'], 3)
      
    desired_order_list = ['deflections', 'saves', 'goals', 'SV%', 'goalie_id']
    for key in desired_order_list:
        goalie_db[goalie][key] = goalie_db[goalie].pop(key)

In [209]:
# Create dataframe with clear information

goalie_df = pd.DataFrame.from_dict(goalie_db, orient='index')
goalie_df = goalie_df.sort_values(by = ['SV%'], ascending=False)
goalie_df


,deflections,saves,goals,SV%,goalie_id
"Romanov, Georgi",3,3,0,1.000,8484312
"Subban, Malcolm",3,3,0,1.000,8476876
"Hellberg, Magnus",7,7,0,1.000,8476433
"Silovs, Arturs",1,1,0,1.000,8481668
"Wallstedt, Jesper",5,5,0,1.000,8482661
...,...,...,...,...,...
"Sogaard, Mads",12,8,4,0.667,8481544
"Hutchinson, Michael",3,2,1,0.667,8474636
"Domingue, Louis",1,0,1,0.000,8475839
"Appleby, Kenneth",1,0,1,0.000,8478965


In [212]:
# Save data to pickle to be accessed by deflection_metrics_visualization.ipynb

goalie_df.to_pickle('2023-24_goalie_deflection_metrics.pickle')